In [16]:
# ============================================================================
# ФАЙЛ: analysis_student_works.py
# ДЕТАЛЬНЫЙ EDA АНАЛИЗ СТУДЕНЧЕСКИХ РАБОТ
# ============================================================================

import os
import re
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# NLP библиотеки
import nltk
from nltk.corpus import stopwords
from langdetect import detect, DetectorFactory, LangDetectException
DetectorFactory.seed = 0
import pdfplumber
from docx import Document
import textract

print("✅ Библиотеки загружены успешно!")

# ============================================================================
# 1. КОНФИГУРАЦИЯ И ПАРАМЕТРЫ АНАЛИЗА
# ============================================================================

# Пути к данным
COURSE_MATERIALS_PATH = Path('course_materials_detailed_analysis.csv')
STUDENT_WORKS_DIR = Path('data/student-works')

# Параметры анализа
STUDENT_WORK_TYPES = ['.pdf', '.docx', '.txt', '.doc']
MAX_PAGES_PER_WORK = 30  # Ограничение на количество страниц для анализа

print("⚙️ Конфигурация загружена")
print(f"Директория студенческих работ: {STUDENT_WORKS_DIR}")

✅ Библиотеки загружены успешно!
⚙️ Конфигурация загружена
Директория студенческих работ: data/student-works


In [12]:
# ============================================================================
# 2. ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ ДЛЯ РАБОТЫ С РАЗНЫМИ ФОРМАТАМИ
# ============================================================================

def extract_text_from_pdf(file_path, max_pages=MAX_PAGES_PER_WORK):
    """Извлечение текста из PDF файла"""
    text = ""
    try:
        with pdfplumber.open(file_path) as pdf:
            for i, page in enumerate(pdf.pages[:max_pages]):
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
    except Exception as e:
        print(f"Ошибка при чтении PDF {file_path.name}: {e}")
    return text

def extract_text_from_docx(file_path):
    """Извлечение текста из DOCX файла"""
    text = ""
    try:
        doc = Document(file_path)
        for paragraph in doc.paragraphs:
            text += paragraph.text + "\n"
    except Exception as e:
        print(f"Ошибка при чтении DOCX {file_path.name}: {e}")
    return text

def extract_text_from_txt(file_path):
    """Извлечение текста из TXT файла"""
    try:
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            text = f.read()
    except Exception as e:
        print(f"Ошибка при чтении TXT {file_path.name}: {e}")
        text = ""
    return text

def extract_text_from_file(file_path):
    """Автоматическое определение формата и извлечение текста"""
    ext = file_path.suffix.lower()

    if ext == '.pdf':
        return extract_text_from_pdf(file_path)
    elif ext == '.docx':
        return extract_text_from_docx(file_path)
    elif ext == '.doc':
        # Для .doc используем textract
        try:
            text = textract.process(file_path).decode('utf-8')
            return text
        except:
            return extract_text_from_txt(file_path)
    elif ext == '.txt':
        return extract_text_from_txt(file_path)
    else:
        print(f"Неподдерживаемый формат файла: {ext}")
        return ""

print("✅ Функции для работы с файлами созданы")

✅ Функции для работы с файлами созданы


In [13]:
# ============================================================================
# 3. СЛОВАРИ И РЕСУРСЫ ДЛЯ АНАЛИЗА СТУДЕНЧЕСКИХ РАБОТ
# ============================================================================

# Загружаем стоп-слова
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

# Создаем расширенный набор стоп-слов для студенческих работ
STOPWORDS_RU = set(stopwords.words('russian'))
STOPWORDS_EN = set(stopwords.words('english'))

STUDENT_STOPWORDS = set()
STUDENT_STOPWORDS.update(STOPWORDS_RU)
STUDENT_STOPWORDS.update(STOPWORDS_EN)
STUDENT_STOPWORDS.update([
    # Общие слова
    'это', 'вот', 'что', 'как', 'так', 'и', 'а', 'но', 'да', 'нет',
    'или', 'ли', 'же', 'бы', 'б', 'ведь', 'вроде', 'дескать', 'мол',
    'ну', 'то', 'все', 'всё', 'еще', 'ещё', 'уже', 'тоже', 'также',
    'либо', 'нибудь', 'какой', 'какая', 'какое', 'какие', 'чей',
    'чья', 'чьё', 'чьи', 'сколько', 'который', 'которая', 'которое',
    'которые', 'никто', 'ничто', 'некий', 'некоторый', 'какой-то',

    # Слова, характерные для студенческих работ
    'работа', 'курсовая', 'реферат', 'эссе', 'исследование', 'анализ',
    'изучение', 'цель', 'задача', 'введение', 'заключение', 'вывод',
    'результат', 'метод', 'методика', 'теоретический', 'практический',
    'студент', 'автор', 'написано', 'приведено', 'рассмотрено',
    'проанализировано', 'исследовано', 'изучено', 'сделано', 'показано',
    'глава', 'параграф', 'раздел', 'часть', 'страница', 'лист',
    'университет', 'институт', 'факультет', 'кафедра', 'преподаватель',
    'руководитель', 'научный', 'учебный', 'учебное', 'пособие',
    'литература', 'источник', 'ссылка', 'библиография', 'список',
    'таблица', 'рисунок', 'график', 'диаграмма', 'приложение',
    'содержание', 'оглавление', 'аннотация', 'рецензия', 'отзыв'
])

# Критерии оценки студенческих работ
EVALUATION_CRITERIA = {
    'structure': ['введение', 'заключение', 'оглавление', 'содержание'],
    'academic': ['источник', 'литература', 'ссылка', 'цитата', 'ссылк'],
    'analysis': ['анализ', 'исследование', 'изучение', 'рассмотрение'],
    'methodology': ['метод', 'методика', 'подход', 'теория'],
    'results': ['результат', 'вывод', 'итог', 'заключение'],
    'formatting': ['таблица', 'рисунок', 'график', 'диаграмма', 'приложение']
}

# Оценочные метрики
EVALUATION_METRICS = {
    'min_words_for_essay': 1500,      # Минимальное количество слов для эссе
    'min_words_for_coursework': 3000, # Минимальное количество слов для курсовой
    'min_references': 5,              # Минимальное количество источников
    'max_special_chars': 0.15,        # Максимальная доля спецсимволов
    'min_lexical_diversity': 0.3,     # Минимальное лексическое разнообразие
}

print("✅ Словари и критерии оценки загружены")

✅ Словари и критерии оценки загружены


In [14]:
# ============================================================================
# 4. КЛАСС ДЛЯ АНАЛИЗА СТУДЕНЧЕСКИХ РАБОТ
# ============================================================================

class StudentWorksAnalyzer:
    """Анализатор студенческих работ с расширенным EDA"""

    def __init__(self, works_dir, course_materials_path=None):
        self.works_dir = Path(works_dir)
        self.works_files = self._find_student_works()
        self.results = []
        self.all_text = ""
        self.course_data = None

        # Загружаем данные материалов курса, если указан путь
        if course_materials_path and Path(course_materials_path).exists():
            self.course_data = pd.read_csv(course_materials_path)
            print(f"✅ Загружены данные материалов курса: {len(self.course_data)} записей")
        else:
            print("⚠️ Данные материалов курса не загружены")

    def _find_student_works(self):
        """Поиск студенческих работ в директории"""
        works = []
        for ext in STUDENT_WORK_TYPES:
            works.extend(list(self.works_dir.glob(f"*{ext}")))
            works.extend(list(self.works_dir.glob(f"**/*{ext}")))  # Рекурсивно

        print(f"📁 Найдено студенческих работ: {len(works)}")
        return works

    def analyze_language(self, text):
        """Анализ языка текста"""
        try:
            return detect(text[:1000])
        except LangDetectException:
            return "unknown"

    def calculate_basic_metrics(self, text):
        """Расчет базовых метрик текста"""
        if not text:
            return {
                'total_chars': 0,
                'total_words': 0,
                'avg_word_length': 0,
                'special_chars_ratio': 0,
                'avg_sentence_length': 0,
                'lexical_diversity': 0,
                'readability_score': 0
            }

        # Основные метрики
        total_chars = len(text)
        words = re.findall(r'\b\w+\b', text.lower())
        total_words = len(words)

        # Средняя длина слова
        avg_word_length = sum(len(word) for word in words) / max(total_words, 1)

        # Доля специальных символов
        special_chars = len(re.findall(r'[^\w\s]', text))
        special_chars_ratio = special_chars / max(total_chars, 1)

        # Средняя длина предложения
        sentences = re.split(r'[.!?]+', text)
        num_sentences = len([s for s in sentences if s.strip()])
        avg_sentence_length = total_words / max(num_sentences, 1)

        # Лексическое разнообразие
        unique_words = set(words)
        lexical_diversity = len(unique_words) / max(total_words, 1) if total_words > 0 else 0

        # Оценка читаемости (упрощенная)
        readability_score = (avg_word_length * 0.5) + (avg_sentence_length * 0.3)

        return {
            'total_chars': total_chars,
            'total_words': total_words,
            'avg_word_length': avg_word_length,
            'special_chars_ratio': special_chars_ratio,
            'avg_sentence_length': avg_sentence_length,
            'lexical_diversity': lexical_diversity,
            'readability_score': readability_score
        }

    def analyze_academic_quality(self, text):
        """Анализ академического качества работы"""
        text_lower = text.lower()
        results = {}

        # Проверяем наличие ключевых структурных элементов
        for criterion, keywords in EVALUATION_CRITERIA.items():
            count = 0
            for keyword in keywords:
                count += len(re.findall(keyword.lower(), text_lower))
            results[f'criterion_{criterion}'] = count

        # Поиск ссылок и источников
        reference_patterns = [
            r'\[[\d\s,]+\]',  # [1], [2, 3]
            r'\(\d{4}\)',     # (2020)
            r'[А-ЯЁA-Z][а-яёa-z]+\sи\sдр\.',  # Иванов и др.
            r'[А-ЯЁA-Z][а-яёa-z]+\set\sal\.',  # Ivanov et al.
        ]

        references_count = 0
        for pattern in reference_patterns:
            references_count += len(re.findall(pattern, text))

        results['references_count'] = references_count

        # Оценка уникальности (простая эвристика)
        sentences = re.split(r'[.!?]+', text)
        unique_sentences = set(s.strip().lower() for s in sentences if len(s.strip()) > 20)
        uniqueness_ratio = len(unique_sentences) / max(len(sentences), 1)
        results['uniqueness_ratio'] = uniqueness_ratio

        return results

    def extract_key_phrases(self, text, top_n=15):
        """Извлечение ключевых фраз из текста"""
        if not text:
            return {}

        text_lower = text.lower()
        text_clean = re.sub(r'[^\w\s]', ' ', text_lower)
        text_clean = re.sub(r'\d+', ' ', text_clean)

        # Извлекаем слова
        words = text_clean.split()
        filtered_words = [
            word for word in words
            if (word not in STUDENT_STOPWORDS and
                len(word) > 3 and
                not word.isspace())
        ]

        # Извлекаем словосочетания (биграммы)
        bigrams = []
        for i in range(len(filtered_words) - 1):
            bigram = f"{filtered_words[i]} {filtered_words[i+1]}"
            if len(bigram.split()) == 2:
                bigrams.append(bigram)

        # Считаем частоту
        word_freq = Counter(filtered_words)
        bigram_freq = Counter(bigrams)

        # Объединяем результаты
        all_phrases = dict(word_freq.most_common(top_n))
        all_phrases.update(dict(bigram_freq.most_common(top_n // 2)))

        return dict(sorted(all_phrases.items(), key=lambda x: x[1], reverse=True)[:top_n])

    def detect_work_type(self, text, filename):
        """Определение типа студенческой работы"""
        text_lower = text.lower()
        filename_lower = filename.lower()

        # По названию файла
        if any(term in filename_lower for term in ['курсовая', 'coursework', 'course_work']):
            return 'курсовая работа'
        elif any(term in filename_lower for term in ['эссе', 'essay']):
            return 'эссе'
        elif any(term in filename_lower for term in ['реферат', 'report']):
            return 'реферат'
        elif any(term in filename_lower for term in ['диплом', 'thesis', 'дипломная']):
            return 'дипломная работа'
        elif any(term in filename_lower for term in ['проект', 'project']):
            return 'проект'

        # По содержанию текста
        if 'курсовая работа' in text_lower or 'курсовой проект' in text_lower:
            return 'курсовая работа'
        elif 'эссе' in text_lower or 'essay' in text_lower:
            return 'эссе'
        elif 'реферат' in text_lower:
            return 'реферат'
        elif 'диплом' in text_lower or 'выпускная квалификационная' in text_lower:
            return 'дипломная работа'
        elif 'проект' in text_lower and 'курсовой' not in text_lower:
            return 'проект'

        # По объему текста
        words_count = len(re.findall(r'\b\w+\b', text_lower))
        if words_count > 5000:
            return 'курсовая работа (предположительно)'
        elif words_count > 1000:
            return 'эссе (предположительно)'
        else:
            return 'не определен'

    def calculate_evaluation_score(self, metrics, academic_metrics):
        """Расчет оценочного балла для работы"""
        score = 0
        max_score = 100

        # 1. Объем работы (30 баллов)
        words = metrics['total_words']
        if words >= EVALUATION_METRICS['min_words_for_coursework']:
            score += 30
        elif words >= EVALUATION_METRICS['min_words_for_essay']:
            score += 20
        elif words >= 500:
            score += 10

        # 2. Лексическое разнообразие (20 баллов)
        lexical_diversity = metrics['lexical_diversity']
        if lexical_diversity >= 0.5:
            score += 20
        elif lexical_diversity >= EVALUATION_METRICS['min_lexical_diversity']:
            score += 15
        elif lexical_diversity >= 0.2:
            score += 10
        else:
            score += 5

        # 3. Академическое качество (30 баллов)
        references = academic_metrics.get('references_count', 0)
        if references >= 10:
            score += 30
        elif references >= EVALUATION_METRICS['min_references']:
            score += 20
        elif references >= 3:
            score += 10

        # 4. Структура работы (20 баллов)
        structure_score = 0
        for key in ['criterion_structure', 'criterion_analysis', 'criterion_results']:
            if academic_metrics.get(key, 0) > 0:
                structure_score += 5

        score += min(structure_score, 20)

        return min(score, max_score)

    def compare_with_course_materials(self, student_keywords, student_metrics):
        """Сравнение студенческой работы с материалами курса"""
        if self.course_data is None or self.course_data.empty:
            return {
                'similarity_score': 0,
                'common_topics': [],
                'course_coverage': 0
            }

        # Извлекаем ключевые слова из материалов курса
        course_keywords = {}
        if 'top_keywords' in self.course_data.columns:
            for keywords_str in self.course_data['top_keywords'].dropna():
                try:
                    # Преобразуем строку в словарь
                    if isinstance(keywords_str, str):
                        # Формат: слово1(10), слово2(5)
                        pairs = keywords_str.split(', ')
                        for pair in pairs:
                            if '(' in pair and ')' in pair:
                                word, count = pair.split('(')
                                count = int(count.replace(')', ''))
                                word = word.strip()
                                course_keywords[word] = course_keywords.get(word, 0) + count
                except:
                    continue

        # Вычисляем сходство по ключевым словам
        common_keywords = set(student_keywords.keys()) & set(course_keywords.keys())
        similarity_score = len(common_keywords) / max(len(set(student_keywords.keys())), 1)

        # Находим общие темы
        common_topics = []
        for keyword in common_keywords:
            student_freq = student_keywords.get(keyword, 0)
            course_freq = course_keywords.get(keyword, 0)
            common_topics.append({
                'keyword': keyword,
                'student_freq': student_freq,
                'course_freq': course_freq
            })

        # Сортируем по частоте у студента
        common_topics.sort(key=lambda x: x['student_freq'], reverse=True)

        # Охват материалов курса
        total_course_keywords = len(course_keywords)
        coverage = len(common_keywords) / max(total_course_keywords, 1)

        return {
            'similarity_score': similarity_score,
            'common_topics': common_topics[:10],  # Топ-10 общих тем
            'course_coverage': coverage,
            'common_keywords_count': len(common_keywords)
        }

    def analyze_all_works(self):
        """Анализ всех студенческих работ"""
        print("\n" + "="*80)
        print("📚 АНАЛИЗ СТУДЕНЧЕСКИХ РАБОТ")
        print("="*80)

        for i, work_file in enumerate(self.works_files, 1):
            print(f"\n📝 Работа {i}/{len(self.works_files)}: {work_file.name}")
            print("-" * 40)

            # Извлечение текста
            text = extract_text_from_file(work_file)
            self.all_text += text + "\n"

            if not text:
                print("  ⚠️ Файл пуст или не может быть прочитан")
                continue

            # Базовые метрики
            metrics = self.calculate_basic_metrics(text)

            # Академическое качество
            academic_metrics = self.analyze_academic_quality(text)

            # Ключевые фразы
            key_phrases = self.extract_key_phrases(text, top_n=15)

            # Определение типа работы
            work_type = self.detect_work_type(text, work_file.name)

            # Язык работы
            language = self.analyze_language(text)

            # Оценочный балл
            evaluation_score = self.calculate_evaluation_score(metrics, academic_metrics)

            # Сравнение с материалами курса
            comparison = self.compare_with_course_materials(key_phrases, metrics)

            # Вывод результатов для этой работы
            print(f"  Тип работы: {work_type}")
            print(f"  Язык: {language}")
            print(f"  Объем: {metrics['total_words']:,} слов")
            print(f"  Лексическое разнообразие: {metrics['lexical_diversity']:.2%}")
            print(f"  Источников: {academic_metrics.get('references_count', 0)}")
            print(f"  Оценочный балл: {evaluation_score}/100")
            print(f"  Сходство с курсом: {comparison['similarity_score']:.2%}")

            if key_phrases:
                top_phrases = list(key_phrases.items())[:5]
                phrases_str = ', '.join([f'{k}({v})' for k, v in top_phrases])
                print(f"  Топ-5 ключевых фраз: {phrases_str}")

            if comparison['common_topics']:
                common_str = ', '.join([f"{item['keyword']}" for item in comparison['common_topics'][:3]])
                print(f"  Общие темы с курсом: {common_str}")

            # Сохранение результатов
            self.results.append({
                'file_name': work_file.name,
                'file_extension': work_file.suffix,
                'file_size_mb': work_file.stat().st_size / (1024 * 1024),
                'work_type': work_type,
                'language': language,
                'total_chars': metrics['total_chars'],
                'total_words': metrics['total_words'],
                'avg_word_length': metrics['avg_word_length'],
                'special_chars_ratio': metrics['special_chars_ratio'],
                'avg_sentence_length': metrics['avg_sentence_length'],
                'lexical_diversity': metrics['lexical_diversity'],
                'readability_score': metrics['readability_score'],
                'key_phrases_count': len(key_phrases),
                'top_key_phrases': dict(list(key_phrases.items())[:10]),
                'references_count': academic_metrics.get('references_count', 0),
                'structure_score': sum(academic_metrics.get(f'criterion_{k}', 0)
                                     for k in ['structure', 'analysis', 'results']),
                'uniqueness_ratio': academic_metrics.get('uniqueness_ratio', 0),
                'evaluation_score': evaluation_score,
                'similarity_with_course': comparison['similarity_score'],
                'course_coverage': comparison['course_coverage'],
                'common_keywords_count': comparison['common_keywords_count'],
                'top_common_topics': [item['keyword'] for item in comparison['common_topics'][:5]]
            })

        # Создаем DataFrame с результатами
        works_df = pd.DataFrame(self.results)

        if not works_df.empty:
            print("\n" + "="*80)
            print("✅ АНАЛИЗ СТУДЕНЧЕСКИХ РАБОТ ЗАВЕРШЕН!")
            print(f"📊 Проанализировано работ: {len(works_df)}")
            print("="*80)
        else:
            print("\n⚠️ Не удалось проанализировать ни одной работы")

        return works_df

print("✅ Класс анализатора студенческих работ создан успешно!")

✅ Класс анализатора студенческих работ создан успешно!


In [18]:
# ============================================================================
# 5. ЗАПУСК АНАЛИЗА И ОСНОВНОЙ EDA
# ============================================================================

# Создаем анализатор
if STUDENT_WORKS_DIR.exists():
    analyzer = StudentWorksAnalyzer(STUDENT_WORKS_DIR, COURSE_MATERIALS_PATH)
    works_df = analyzer.analyze_all_works()
else:
    print(f"⚠️ Директория {STUDENT_WORKS_DIR} не найдена!")
    works_df = pd.DataFrame()

if not works_df.empty:
    # ============================================================================
    # 6. СВОДНАЯ СТАТИСТИКА ПО СТУДЕНЧЕСКИМ РАБОТАМ
    # ============================================================================

    print("\n" + "="*80)
    print("📊 СВОДНАЯ СТАТИСТИКА ПО СТУДЕНЧЕСКИМ РАБОТАМ")
    print("="*80)

    # Основные метрики
    print("\n📈 ОСНОВНЫЕ МЕТРИКИ:")
    print("-" * 40)
    print(f"Всего работ: {len(works_df)}")
    print(f"Общий объем текста: {works_df['total_words'].sum():,} слов")
    print(f"Средний объем работы: {works_df['total_words'].mean():,.0f} слов")
    print(f"Медианный объем: {works_df['total_words'].median():,.0f} слов")
    print(f"Средняя оценка: {works_df['evaluation_score'].mean():.1f}/100")
    print(f"Среднее сходство с курсом: {works_df['similarity_with_course'].mean():.2%}")

    # Распределение по типам работ
    if 'work_type' in works_df.columns:
        print(f"\n📚 РАСПРЕДЕЛЕНИЕ ПО ТИПАМ РАБОТ:")
        print("-" * 40)
        type_counts = works_df['work_type'].value_counts()
        for work_type, count in type_counts.items():
            percentage = (count / len(works_df)) * 100
            print(f"  {work_type}: {count} работ ({percentage:.1f}%)")

    # Распределение по языкам
    if 'language' in works_df.columns:
        print(f"\n🌍 РАСПРЕДЕЛЕНИЕ ПО ЯЗЫКАМ:")
        print("-" * 40)
        lang_counts = works_df['language'].value_counts()
        for lang, count in lang_counts.items():
            percentage = (count / len(works_df)) * 100
            print(f"  {lang}: {count} работ ({percentage:.1f}%)")

    # Анализ качества
    print(f"\n🎯 АНАЛИЗ КАЧЕСТВА РАБОТ:")
    print("-" * 40)

    # Оценки
    evaluation_stats = works_df['evaluation_score'].describe()
    print(f"  Минимальная оценка: {evaluation_stats['min']:.1f}")
    print(f"  Средняя оценка: {evaluation_stats['mean']:.1f}")
    print(f"  Максимальная оценка: {evaluation_stats['max']:.1f}")

    # Лучшие и худшие работы
    if len(works_df) > 0:
        best_work = works_df.loc[works_df['evaluation_score'].idxmax()]
        worst_work = works_df.loc[works_df['evaluation_score'].idxmin()]

        print(f"\n🏆 ЛУЧШАЯ РАБОТА:")
        print(f"  Файл: {best_work['file_name']}")
        print(f"  Оценка: {best_work['evaluation_score']:.1f}/100")
        print(f"  Объем: {best_work['total_words']:,} слов")

        print(f"\n⚠️ ХУДШАЯ РАБОТА:")
        print(f"  Файл: {worst_work['file_name']}")
        print(f"  Оценка: {worst_work['evaluation_score']:.1f}/100")
        print(f"  Объем: {worst_work['total_words']:,} слов")

    # Анализ сходства с курсом
    print(f"\n📚 СХОДСТВО С МАТЕРИАЛАМИ КУРСА:")
    print("-" * 40)
    similarity_stats = works_df['similarity_with_course'].describe()
    print(f"  Среднее сходство: {similarity_stats['mean']:.2%}")
    print(f"  Минимальное сходство: {similarity_stats['min']:.2%}")
    print(f"  Максимальное сходство: {similarity_stats['max']:.2%}")

    # Работы с высоким сходством
    high_similarity = works_df[works_df['similarity_with_course'] > 0.5]
    if len(high_similarity) > 0:
        print(f"  Работ с высоким сходством (>50%): {len(high_similarity)}")

    # Работы с низким сходством
    low_similarity = works_df[works_df['similarity_with_course'] < 0.1]
    if len(low_similarity) > 0:
        print(f"  Работ с низким сходством (<10%): {len(low_similarity)}")

    print("\n" + "="*80)
    print("✅ СВОДНЫЙ АНАЛИЗ ЗАВЕРШЕН!")
    print("="*80)

📁 Найдено студенческих работ: 16
⚠️ Данные материалов курса не загружены

📚 АНАЛИЗ СТУДЕНЧЕСКИХ РАБОТ

📝 Работа 1/16: Эссе_соразмерный_город_Пушкарева_Дарья.docx
----------------------------------------
  Тип работы: эссе
  Язык: ru
  Объем: 1,347 слов
  Лексическое разнообразие: 56.27%
  Источников: 0
  Оценочный балл: 30/100
  Сходство с курсом: 0.00%
  Топ-5 ключевых фраз: город(15), города(9), жизни(6), понимала(6), круг(4)


KeyError: 'common_keywords_count'